In [1]:
import csv
from PIL import Image
import pandas as pd
import math
from matplotlib import pyplot as plt

In [2]:
# Initializing control variables
csv_path = "/home/felipe/Projects/machineLearning/datasets/food-101/images/test/results.csv"

right_predictions = 0
wrong_predictions = 0
elements = 0

In [3]:
# Initializing arrays
y_index = []
y_label = []

y_pred_index = []
y_pred_label = []

images_paths = []

In [4]:
# Reading CSV content
with open(csv_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')

    for row in csv_reader:
        elements += 1

        y = row[1]
        y_pred = row[3]

        y_index.append(row[0])
        y_label.append(row[1])
        y_pred_index.append(row[2])
        y_pred_label.append(row[3])
        
        images_paths.append(row[4])

        if y == y_pred:
            right_predictions += 1
        else:
            wrong_predictions += 1

    csv_file.close()

In [5]:
print("Amount of elements: " + str(elements))
print("Right predictions: " + str(right_predictions))
print("Wrong predictions: " + str(wrong_predictions))

Amount of elements: 20200
Right predictions: 13425
Wrong predictions: 6775


In [6]:
df = pd.DataFrame({"prediction": y_pred_label, "actual": y_label})


In [7]:
dataJsonString = df.to_json(orient='records')

In [8]:
image_miniature_width = 64
image_miniature_height = 64
image_miniature_size = image_miniature_width, image_miniature_height

images_amount = len(images_paths)

num_sprites_x = int(math.sqrt(images_amount) + 0.5)
num_sprites_y = num_sprites_x if (num_sprites_x ** 2) == images_amount else num_sprites_x + 1

atlas_width = image_miniature_width * num_sprites_x
atlas_height = image_miniature_height * num_sprites_y

atlas  = Image.new("RGB", (atlas_width, atlas_height), (0, 0, 0))

for i in range(num_sprites_y): 
    for j in range(num_sprites_x):
        image_index = num_sprites_x * i + j
        
        if image_index < images_amount:
            image_path = images_paths[image_index]
            image = Image.open(image_path).resize(image_miniature_size)

            x = j*image_miniature_width
            y = i*image_miniature_height

            atlas.paste(image, (x, y))
                        
plt.figure(figsize=(15,15))
plt.imshow(atlas)

atlas.save("atlas.jpg", "JPEG")

In [12]:
from IPython.core.display import display, HTML

HTML_TEMPLATE = """
        <head>
        <link rel="import" href="./facets-jupyter.html"></link>
        </head>
        <facets-dive id="elem" height="1200" sprite-image-width="64" sprite-image-height="64" atlas-url="atlas.jpg"></facets-dive>
        <script>
          var data = JSON.parse('{jsonstr}');
                    
          var fd = document.querySelector("#elem");
          fd.data = data;
          fd['verticalFacet'] = 'prediction';
          fd['verticalBuckets'] = 101;
          fd['horizontalFacet'] = 'actual';
          fd['horizontalBuckets'] = 101;
          fd['colorBy'] = 'actual';
          
          
        </script>"""
html = HTML_TEMPLATE.format(jsonstr=dataJsonString)
display(HTML(html))